# 라이브러리 로드

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import gc
from tensorflow.keras.backend import clear_session

from matplotlib import rcParams, pyplot as plt

import numpy as np
import pandas as pd
import random as rn
import re

from pathlib import Path
import warnings

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence

from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, GlobalMaxPooling1D, Conv1D, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.optimizers import Adam
import warnings 

In [3]:
rcParams['figure.figsize'] = (16, 8)
plt.style.use('fivethirtyeight')
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
warnings.simplefilter('ignore')

# 학습데이터 로드

In [4]:
seed = 2020
# np.random.seed(seed)
# rn.seed(seed)
# tf.random.set_seed(seed)
# session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,
#                              inter_op_parallelism_threads=1)
# sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
# tf.compat.v1.keras.backend.set_session(sess)

In [5]:
data_dir = Path('../data/dacon-novel-author-classification')
feature_dir = Path('../build/feature')
val_dir = Path('../build/val')
tst_dir = Path('../build/tst')
sub_dir = Path('../build/sub')

trn_file = data_dir / 'train.csv'
tst_file = data_dir / 'test_x.csv'
sample_file = data_dir / 'sample_submission.csv'

target_col = 'author'
n_fold = 5
n_class = 5

In [6]:
algorithm_name = 'cnn'
feature_name = 'feature'
model_name = f'{algorithm_name}_{feature_name}'

feature_Ver1_file = feature_dir / f'{feature_name}_Ver1.csv'
feature_Ver2_file = feature_dir / f'{feature_name}_Ver2.csv'
feature_Ver3_file = feature_dir / f'{feature_name}_Ver3.csv'
feature_Ver4_file = feature_dir / f'{feature_name}_Ver4.csv'


cnn_oof_pred_ver1_file = val_dir / f'{model_name}_oof_pred_ver1.csv'
cnn_oof_pred_ver2_file = val_dir / f'{model_name}_oof_pred_ver2.csv'
cnn_oof_pred_ver3_file = val_dir / f'{model_name}_oof_pred_ver3.csv'
cnn_oof_pred_ver4_file = val_dir / f'{model_name}_oof_pred_ver4.csv'


cnn_test_pred_ver1_file = tst_dir / f'{model_name}_test_pred_ver1.csv'
cnn_test_pred_ver2_file = tst_dir / f'{model_name}_test_pred_ver2.csv'
cnn_test_pred_ver3_file = tst_dir / f'{model_name}_test_pred_ver3.csv'
cnn_test_pred_ver4_file = tst_dir / f'{model_name}_test_pred_ver4.csv'


cnn_submission_ver1_file = sub_dir / f'{model_name}_submission_Ver1.csv'
cnn_submission_ver2_file = sub_dir / f'{model_name}_submission_Ver2.csv'
cnn_submission_ver3_file = sub_dir / f'{model_name}_submission_Ver3.csv'
cnn_submission_ver4_file = sub_dir / f'{model_name}_submission_Ver4.csv'

## Ver 1 데이터 로드

In [7]:
dataset = pd.read_csv(feature_Ver1_file, index_col=0)
print(dataset.shape)
dataset.head()

(74496, 2)


,text,author
index,,
0,he was almost choking there was so much so muc...,3.0
1,your sister asked for it i suppose,2.0
2,she was engaged one day as she walked in peru...,1.0
3,the captain was in the porch keeping himself c...,4.0
4,have mercy gentlemen odin flung up his hands d...,3.0


In [8]:
# train set
Ver1_X = dataset.loc[dataset[target_col] != -1 , :]
Ver1_X.drop(columns=target_col,inplace=True,axis=1)
Ver1_y = dataset.loc[dataset[target_col] != -1, target_col]
Ver1_y.astype(int)

# test set
Ver1_test = dataset.loc[dataset[target_col] == -1, :]
Ver1_test.drop(columns=target_col, inplace=True,axis=1)

print(Ver1_X.shape, Ver1_y.shape, Ver1_test.shape)

(54879, 1) (54879,) (19617, 1)


## Ver 2 데이터 로드

In [9]:
dataset = pd.read_csv(feature_Ver2_file, index_col=0)
print(dataset.shape)
dataset.head()

(74496, 2)


,text,author
index,,
0,almost choking much much wanted say strange ex...,3.0
1,sister asked suppose,2.0
2,engaged one day walked perusing janes last let...,1.0
3,captain porch keeping carefully way treacherou...,4.0
4,mercy gentlemen odin flung hands dont write an...,3.0


In [10]:
# train set
Ver2_X = dataset.loc[dataset[target_col] != -1 , :]
Ver2_X.drop(columns=target_col,inplace=True,axis=1)
Ver2_y = dataset.loc[dataset[target_col] != -1, target_col]
Ver2_y.astype(int)

# test set
Ver2_test = dataset.loc[dataset[target_col] == -1, :]
Ver2_test.drop(columns=target_col, inplace=True,axis=1)

print(Ver2_X.shape, Ver2_y.shape, Ver2_test.shape)

(54879, 1) (54879,) (19617, 1)


## Ver3 데이터 로드

In [11]:
dataset = pd.read_csv(feature_Ver3_file, index_col=0)
print(dataset.shape)
dataset.head()

(74496, 2)


,text,author
index,,
0,almost choking much much wanted say strange ex...,3.0
1,sister asked suppose,2.0
2,engaged day walked perusing janes last letter ...,1.0
3,captain porch keeping himself carefully out wa...,4.0
4,mercy gentlemen flung up hands dont write anyw...,3.0


In [12]:
# train set
Ver3_X = dataset.loc[dataset[target_col] != -1 , :]
Ver3_X.drop(columns=target_col,inplace=True,axis=1)
Ver3_y = dataset.loc[dataset[target_col] != -1, target_col]
Ver3_y.astype(int)

# test set
Ver3_test = dataset.loc[dataset[target_col] == -1, :]
Ver3_test.drop(columns=target_col, inplace=True,axis=1)

print(Ver3_X.shape, Ver3_y.shape, Ver3_test.shape)

(54879, 1) (54879,) (19617, 1)


## Ver4 데이터 로드

In [13]:
dataset = pd.read_csv(feature_Ver4_file, index_col=0)
print(dataset.shape)
dataset.head()

(74496, 2)


,text,author
index,,
0,he almost choking there so much so much he wan...,3.0
1,sister asked for it suppose,2.0
2,she engaged one day she walked perusing janes ...,1.0
3,captain porch keeping himself carefully out wa...,4.0
4,have mercy gentlemen odin flung up hands dont ...,3.0


In [14]:
# train set
Ver4_X = dataset.loc[dataset[target_col] != -1 , :]
Ver4_X.drop(columns=target_col,inplace=True,axis=1)
Ver4_y = dataset.loc[dataset[target_col] != -1, target_col]
Ver4_y.astype(int)

# test set
Ver4_test = dataset.loc[dataset[target_col] == -1, :]
Ver4_test.drop(columns=target_col, inplace=True,axis=1)

print(Ver4_X.shape, Ver4_y.shape, Ver4_test.shape)

(54879, 1) (54879,) (19617, 1)


# 모델 학습

In [15]:
#파라미터 설정
vocab_size = 30000
embedding_dim = 128
max_length = 500
padding_type='post'
#oov_tok = "<OOV>"

In [16]:
def get_model():
    model = Sequential([
        Embedding(vocab_size, embedding_dim, input_length=max_length),
        Dropout(.5),
        Conv1D(128, 7, padding="valid", activation="relu", strides=3),
        Conv1D(128, 7, padding="valid", activation="relu", strides=3),    
        GlobalMaxPooling1D(),
        Dense(128, activation='relu'),
        Dropout(.5),
        Dense(n_class, activation='softmax')
    ])
    
    # compile model
    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(learning_rate=.0003))
    return model

In [17]:
datasets = [ (Ver1_X, Ver1_test, Ver1_y), (Ver2_X, Ver2_test, Ver2_y),
            (Ver3_X, Ver3_test, Ver3_y), (Ver4_X, Ver4_test, Ver3_y)]

cnn_oof_preds = []
cnn_test_preds = []

cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

for number ,(X, test, y) in enumerate(datasets,1):
    print(f'start : {number}')
    # 토큰화
    X_train = np.array([x for x in X['text']])
    X_test = np.array([x for x in test['text']])
    y = np.array(y.values)
    
    tokenizer = Tokenizer(num_words = vocab_size)
    tokenizer.fit_on_texts(X_train)
    word_index = tokenizer.word_index
    
    # 시퀸스화 + 패딩
    train_sequences = tokenizer.texts_to_sequences(X_train)
    test_sequences = tokenizer.texts_to_sequences(X_test)
    trn = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)
    tst = pad_sequences(test_sequences, padding=padding_type, maxlen=max_length)
    print(trn.shape, tst.shape)
    
    # oof , test 저장
    cnn_oof_pred = np.zeros((trn.shape[0], n_class))
    cnn_test_pred = np.zeros((tst.shape[0], n_class))
    
    for i, (i_trn, i_val) in enumerate(cv.split(trn, y), 1):
        print(f'traing model for CV #{i}')
        clf = get_model()
        
        es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3,
                       verbose=1, mode='min', baseline=None, restore_best_weights=True)
        
        rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                            patience=3, min_lr=1e-6, mode='min', verbose=1)
        
        clf.fit(trn[i_trn], 
            to_categorical(y[i_trn]),
            validation_data=(trn[i_val], to_categorical(y[i_val])),
            epochs=200,
            batch_size=1024,
            callbacks=[es, rlr])
        
        cnn_oof_pred[i_val, :] = clf.predict(trn[i_val])
        cnn_test_pred += clf.predict(tst) / n_fold
        
        del clf
        clear_session()
        gc.collect()
    
    cnn_oof_preds.append(cnn_oof_pred)
    cnn_test_preds.append(cnn_test_pred)
        
    print(f'end : {number}')

start : 1
(54879, 500) (19617, 500)
traing model for CV #1
Epoch 1/200
43/43 [==============================] - 7s 172ms/step - loss: 1.5767 - val_loss: 1.5739
Epoch 2/200
43/43 [==============================] - 7s 160ms/step - loss: 1.5650 - val_loss: 1.5606
Epoch 3/200
43/43 [==============================] - 7s 161ms/step - loss: 1.5241 - val_loss: 1.4717
Epoch 4/200
43/43 [==============================] - 7s 160ms/step - loss: 1.4192 - val_loss: 1.3082
Epoch 5/200
43/43 [==============================] - 7s 161ms/step - loss: 1.2015 - val_loss: 1.0807
Epoch 6/200
43/43 [==============================] - 7s 160ms/step - loss: 1.0125 - val_loss: 0.9669
Epoch 7/200
43/43 [==============================] - 7s 161ms/step - loss: 0.8909 - val_loss: 0.9018
Epoch 8/200
43/43 [==============================] - 7s 159ms/step - loss: 0.8013 - val_loss: 0.8659
Epoch 9/200
43/43 [==============================] - 7s 159ms/step - loss: 0.7313 - val_loss: 0.8385
Epoch 10/200
43/43 [============

43/43 [==============================] - 7s 160ms/step - loss: 1.5379 - val_loss: 1.4896
Epoch 4/200
43/43 [==============================] - 7s 159ms/step - loss: 1.4372 - val_loss: 1.3684
Epoch 5/200
43/43 [==============================] - 7s 159ms/step - loss: 1.3099 - val_loss: 1.2728
Epoch 6/200
43/43 [==============================] - 7s 160ms/step - loss: 1.1811 - val_loss: 1.1533
Epoch 7/200
43/43 [==============================] - 7s 160ms/step - loss: 1.0193 - val_loss: 1.0271
Epoch 8/200
43/43 [==============================] - 7s 159ms/step - loss: 0.8898 - val_loss: 0.9546
Epoch 9/200
43/43 [==============================] - 7s 159ms/step - loss: 0.7848 - val_loss: 0.9100
Epoch 10/200
43/43 [==============================] - 7s 160ms/step - loss: 0.6949 - val_loss: 0.8846
Epoch 11/200
43/43 [==============================] - 7s 159ms/step - loss: 0.6281 - val_loss: 0.8686
Epoch 12/200
43/43 [==============================] - 7s 160ms/step - loss: 0.5715 - val_loss: 0.8613

43/43 [==============================] - 7s 159ms/step - loss: 0.6180 - val_loss: 0.8322
Epoch 13/200
43/43 [==============================] - 7s 161ms/step - loss: 0.5777 - val_loss: 0.8247
Epoch 14/200
43/43 [==============================] - 7s 160ms/step - loss: 0.5364 - val_loss: 0.8166
Epoch 15/200
43/43 [==============================] - 7s 159ms/step - loss: 0.4903 - val_loss: 0.8160
Epoch 16/200
43/43 [==============================] - 7s 161ms/step - loss: 0.4501 - val_loss: 0.8180
Epoch 17/200
43/43 [==============================] - 7s 160ms/step - loss: 0.4097 - val_loss: 0.8288
Epoch 00017: early stopping
traing model for CV #4
Epoch 1/200
43/43 [==============================] - 7s 164ms/step - loss: 1.5767 - val_loss: 1.5725
Epoch 2/200
43/43 [==============================] - 7s 160ms/step - loss: 1.5653 - val_loss: 1.5479
Epoch 3/200
43/43 [==============================] - 7s 160ms/step - loss: 1.5064 - val_loss: 1.4522
Epoch 4/200
43/43 [============================

In [18]:
for i,j in enumerate(cnn_oof_preds,1):
    print(f'ver{i} logloss = {log_loss(pd.get_dummies(y),j):8.4f}')
    print(f'ver{i} accuracy = {accuracy_score(y, np.argmax(j,axis=1))*100:8.4f}')

ver1 logloss =   0.7930
ver1 logloss =  70.9925
ver2 logloss =   0.8967
ver2 logloss =  66.5391
ver3 logloss =   0.9273
ver3 logloss =  66.9564
ver4 logloss =   0.7945
ver4 logloss =  71.5611


# 제출 파일 및 기타 파일 생성

In [19]:
# submission 파일 생성

sub = pd.read_csv(sample_file,index_col=0)

# Ver1 
sub[sub.columns] = cnn_test_preds[0]
sub.to_csv(cnn_submission_ver1_file)

# Ver2
sub[sub.columns] = cnn_test_preds[1]
sub.to_csv(cnn_submission_ver2_file)

# Ver3
sub[sub.columns] = cnn_test_preds[2]
sub.to_csv(cnn_submission_ver3_file)
           
# Ver4
sub[sub.columns] = cnn_test_preds[3]
sub.to_csv(cnn_submission_ver4_file)

In [20]:
# cnn_oof_pred 파일 생성

# Ver1
np.savetxt(cnn_oof_pred_ver1_file, cnn_oof_preds[0],fmt='%.18f', delimiter=',')

# Ver2
np.savetxt(cnn_oof_pred_ver2_file, cnn_oof_preds[1],fmt='%.18f', delimiter=',')

# Ver3
np.savetxt(cnn_oof_pred_ver3_file, cnn_oof_preds[2],fmt='%.18f', delimiter=',')

# Ver4
np.savetxt(cnn_oof_pred_ver4_file, cnn_oof_preds[3],fmt='%.18f', delimiter=',')

In [21]:
# cnn_test_pred 파일 생성

# Ver1
np.savetxt(cnn_test_pred_ver1_file, cnn_test_preds[0],fmt='%.18f', delimiter=',')

# Ver2
np.savetxt(cnn_test_pred_ver2_file, cnn_test_preds[1],fmt='%.18f', delimiter=',')

# Ver3
np.savetxt(cnn_test_pred_ver3_file, cnn_test_preds[2],fmt='%.18f', delimiter=',')

# Ver4
np.savetxt(cnn_test_pred_ver4_file, cnn_test_preds[3],fmt='%.18f', delimiter=',')